<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/Hybrid%20LR-Bi-LSTM%20for%205%20class%20classifications%20based%20on%20WSN-DS%20dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Hybrid LR-Bi-LSTM for 5 class classifications based on WSN-DS dataset**

In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout, Flatten

In [4]:
# Load dataset
dataset = pd.read_csv("dataset_WSN-DS.csv")
dataset = dataset.dropna() # Remove missing values
X = dataset.drop(['Class'], axis=1)
y = dataset['Class']

### **Data Preprocessing**

In [5]:
# Encode labels
encoder = LabelEncoder()
y = encoder.fit_transform(dataset['Class'])

In [6]:
# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(dataset.drop(['Class'], axis=1))

In [7]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### **Hybrid combination of LR-Bi-LSTM for 5 class classifications**

In [8]:
# Train logistic regression model
lr_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)
lr_acc = accuracy_score(y_test, lr_pred)
print("Logistic Regression Accuracy:", lr_acc)
print(classification_report(y_test, lr_pred))

Logistic Regression Accuracy: 0.9718815475157808
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     67965
           1       0.71      0.64      0.67      2985
           2       0.66      0.75      0.71      2043
           3       0.98      0.93      0.95      1309
           4       0.87      0.91      0.89       631

    accuracy                           0.97     74933
   macro avg       0.84      0.85      0.84     74933
weighted avg       0.97      0.97      0.97     74933



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
# Train Bi-LSTM model
num_classes = len(np.unique(y))
input_dim = X_train.shape[1]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
lstm_model = Sequential()
lstm_model.add(Bidirectional(LSTM(64, input_shape=(1, input_dim), activation='relu', return_sequences=True)))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(32, activation='relu'))
lstm_model.add(Dense(5, activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
y_train_onehot = pd.get_dummies(y_train).values
lstm_model.fit(X_train, y_train_onehot, epochs=15, batch_size=32, verbose=0)
lstm_pred = lstm_model.predict(X_test)
lstm_pred = np.argmax(lstm_pred, axis=1)
lstm_acc = accuracy_score(y_test, lstm_pred)
print("Bi-LSTM Accuracy:", lstm_acc)
print(classification_report(y_test, lstm_pred))

2342/2342 [==============================] - 5s 2ms/step
Bi-LSTM Accuracy: 0.9882027944964168
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     67965
           1       0.90      0.85      0.88      2985
           2       0.81      0.88      0.84      2043
           3       0.99      0.93      0.96      1309
           4       0.90      1.00      0.95       631

    accuracy                           0.99     74933
   macro avg       0.92      0.93      0.93     74933
weighted avg       0.99      0.99      0.99     74933



In [10]:
# Combine predictions
lr_probs = lr_model.predict_proba(np.reshape(X_test, (X_test.shape[0], -1)))
lstm_probs = lstm_model.predict(X_test)
combined_probs = (lr_probs + lstm_probs) / 2
combined_pred = np.argmax(combined_probs, axis=1)
combined_acc = accuracy_score(y_test, combined_pred)
print("Combined Accuracy:", combined_acc)
print(classification_report(y_test, combined_pred))

2342/2342 [==============================] - 4s 2ms/step
Combined Accuracy: 0.9876156032722565
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     67965
           1       0.90      0.83      0.87      2985
           2       0.80      0.90      0.84      2043
           3       0.99      0.94      0.96      1309
           4       0.90      0.99      0.94       631

    accuracy                           0.99     74933
   macro avg       0.92      0.93      0.92     74933
weighted avg       0.99      0.99      0.99     74933

